<a href="https://colab.research.google.com/github/RylieWeaver/NLP/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.4 Lab Assignment #3: Word embeddings and sentiment analysis



## Section 0: Setup

In [ ]:
!pip install nltk

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=cff7aaff0c8f5b9d206b62424836554321b73bf21e829fb9641784811f02cca7
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
# The imports needed for this notebook
import numpy as np
import pandas as pd
import ast

In [ ]:
# Setup access to my Google Drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=False)

Mounted at /content/drive


In [ ]:
import gensim

In [ ]:
import nltk

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import contractions
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Section 1: Create the Cleaned Reviews

Create a cleaned review (as a string) column with lemmas for each row of the data provided.

Similar to Step 6 in Lab Assignment 2.

### Read the CSV File

In [ ]:
# Travel to Lab2 Directory
%cd "/content/drive/MyDrive/NLP/Lab3"

/content/drive/MyDrive/NLP/Lab3


In [ ]:
%ls

'NLP Lab3.ipynb'   Restaurant_reviews_R.csv


In [ ]:
column_names = ['Rating', 'Date', 'Review', 'Business', 'City', 'State', 'Zip', 'Business Rating Score']
reviews = pd.read_csv('Restaurant_reviews_R.csv', header=None, names=column_names)

In [ ]:
# Initialize clean dataframe with column to be cleaned (must do deepcopy)

"""Note that I call the new dataframe and column clean, but they are really
Initializations to be cleaned"""

reviews_clean = reviews.copy()
reviews_clean['Cleaned Review'] = reviews_clean['Review']

### Contraction Expansion

In [ ]:
# Define the contraction expansion function
def expand_contractions(text):
  expanded_text = contractions.fix(text)
  return expanded_text

In [ ]:
# Test the contraction expansion
column = reviews_clean['Cleaned Review']
test = column[10]
print(test)
print("----------------------------------------------------------------------")
print(expand_contractions(test))

Shame on me for taking so long to get here! I walk & notice signs saying, "Sue says..." I'm already liking this place. A woman takes my order & I ask if she is the famous Sue. She says, "I'm Sue, but not I'm not famous." 

I get a hot Italian. It was perfect. I like the bun a little toasted so it doesn't get soggy with the oil. I'll be back!
----------------------------------------------------------------------
Shame on me for taking so long to get here! I walk & notice signs saying, "Sue says..." I am already liking this place. A woman takes my order & I ask if she is the famous Sue. She says, "I am Sue, but not I am not famous." 

I get a hot Italian. It was perfect. I like the bun a little toasted so it does not get soggy with the oil. I will be back!


In [ ]:
# Apply contraction expansion to the DataFrame
reviews_clean['Cleaned Review'] = reviews_clean['Cleaned Review'].apply(expand_contractions)

In [ ]:
# Check expansion
reviews_clean.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...


### Tokenize

In [ ]:
def tokenize(text):
  tokenized = word_tokenize(text)
  return tokenized

In [ ]:
# Test the tokenization
column = reviews_clean['Cleaned Review']
test = column[10]
print(test)
print("----------------------------------------------------------------------")
print(tokenize(test))

Shame on me for taking so long to get here! I walk & notice signs saying, "Sue says..." I am already liking this place. A woman takes my order & I ask if she is the famous Sue. She says, "I am Sue, but not I am not famous." 

I get a hot Italian. It was perfect. I like the bun a little toasted so it does not get soggy with the oil. I will be back!
----------------------------------------------------------------------
['Shame', 'on', 'me', 'for', 'taking', 'so', 'long', 'to', 'get', 'here', '!', 'I', 'walk', '&', 'notice', 'signs', 'saying', ',', '``', 'Sue', 'says', '...', "''", 'I', 'am', 'already', 'liking', 'this', 'place', '.', 'A', 'woman', 'takes', 'my', 'order', '&', 'I', 'ask', 'if', 'she', 'is', 'the', 'famous', 'Sue', '.', 'She', 'says', ',', '``', 'I', 'am', 'Sue', ',', 'but', 'not', 'I', 'am', 'not', 'famous', '.', "''", 'I', 'get', 'a', 'hot', 'Italian', '.', 'It', 'was', 'perfect', '.', 'I', 'like', 'the', 'bun', 'a', 'little', 'toasted', 'so', 'it', 'does', 'not', 'get',

In [ ]:
# Apply tokenization to the DataFrame
reviews_clean['Cleaned Review'] = reviews_clean['Cleaned Review'].apply(tokenize)

In [ ]:
# Check tokenization
reviews_clean.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[its, the, best, pizza, hut, i, ', v, found, i..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[This, place, is, an, interesting, combo, ., T..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[Pizza, Hut, is, great, !, You, get, huge, piz..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[Always, a, great, place, to, taste, some, tea..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[Delish, !, Stopped, in, on, a, random, Sunday..."


### Create POS tags

In [ ]:
def pos_tag(tokens):
    tagged = nltk.pos_tag(tokens)
    return tagged

In [ ]:
# Test the tokenization
column = reviews_clean['Cleaned Review']
test = column[10]
print(test)
print("----------------------------------------------------------------------")
print(pos_tag(test))

['Shame', 'on', 'me', 'for', 'taking', 'so', 'long', 'to', 'get', 'here', '!', 'I', 'walk', '&', 'notice', 'signs', 'saying', ',', '``', 'Sue', 'says', '...', "''", 'I', 'am', 'already', 'liking', 'this', 'place', '.', 'A', 'woman', 'takes', 'my', 'order', '&', 'I', 'ask', 'if', 'she', 'is', 'the', 'famous', 'Sue', '.', 'She', 'says', ',', '``', 'I', 'am', 'Sue', ',', 'but', 'not', 'I', 'am', 'not', 'famous', '.', "''", 'I', 'get', 'a', 'hot', 'Italian', '.', 'It', 'was', 'perfect', '.', 'I', 'like', 'the', 'bun', 'a', 'little', 'toasted', 'so', 'it', 'does', 'not', 'get', 'soggy', 'with', 'the', 'oil', '.', 'I', 'will', 'be', 'back', '!']
----------------------------------------------------------------------
[('Shame', 'NN'), ('on', 'IN'), ('me', 'PRP'), ('for', 'IN'), ('taking', 'VBG'), ('so', 'RB'), ('long', 'RB'), ('to', 'TO'), ('get', 'VB'), ('here', 'RB'), ('!', '.'), ('I', 'PRP'), ('walk', 'VBP'), ('&', 'CC'), ('notice', 'JJ'), ('signs', 'NNS'), ('saying', 'VBG'), (',', ','), ('

In [ ]:
# Apply tokenization to the DataFrame
reviews_clean['Cleaned Review'] = reviews_clean['Cleaned Review'].apply(pos_tag)

In [ ]:
# Check tokenization
reviews_clean.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[(its, PRP$), (the, DT), (best, JJS), (pizza, ..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[(This, DT), (place, NN), (is, VBZ), (an, DT),..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[(Pizza, NNP), (Hut, NNP), (is, VBZ), (great, ..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[(Always, VBZ), (a, DT), (great, JJ), (place, ..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[(Delish, JJ), (!, .), (Stopped, VBN), (in, IN..."


### Lemmatization

In [ ]:
# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Function to convert nltk POS tags to WordNet POS tags
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return 'a'  # adjective
    elif nltk_tag.startswith('V'):
        return 'v'  # verb
    elif nltk_tag.startswith('N'):
        return 'n'  # noun
    elif nltk_tag.startswith('R'):
        return 'r'  # adverb
    else:
        return None

In [ ]:
# Function to lemmatize already tokenized and POS-tagged tokens
def lemmatize(tagged_tokens):
    lemmatized_tokens = []
    for word, tag in tagged_tokens:
        wordnet_tag = nltk_tag_to_wordnet_tag(tag)
        if wordnet_tag is None:
            # If there is no available tag, append the token as is
            lemmatized_tokens.append(word)
        else:
            # Else use the tag to lemmatize the token
            lemmatized_tokens.append(lemmatizer.lemmatize(word, wordnet_tag))
    return lemmatized_tokens

In [ ]:
# Test the tokenization
column = reviews_clean['Cleaned Review']
test = column[10]
print(test)
print("----------------------------------------------------------------------")
print(lemmatize(test))

[('Shame', 'NN'), ('on', 'IN'), ('me', 'PRP'), ('for', 'IN'), ('taking', 'VBG'), ('so', 'RB'), ('long', 'RB'), ('to', 'TO'), ('get', 'VB'), ('here', 'RB'), ('!', '.'), ('I', 'PRP'), ('walk', 'VBP'), ('&', 'CC'), ('notice', 'JJ'), ('signs', 'NNS'), ('saying', 'VBG'), (',', ','), ('``', '``'), ('Sue', 'NNP'), ('says', 'VBZ'), ('...', ':'), ("''", "''"), ('I', 'PRP'), ('am', 'VBP'), ('already', 'RB'), ('liking', 'VBG'), ('this', 'DT'), ('place', 'NN'), ('.', '.'), ('A', 'DT'), ('woman', 'NN'), ('takes', 'VBZ'), ('my', 'PRP$'), ('order', 'NN'), ('&', 'CC'), ('I', 'PRP'), ('ask', 'VBP'), ('if', 'IN'), ('she', 'PRP'), ('is', 'VBZ'), ('the', 'DT'), ('famous', 'JJ'), ('Sue', 'NNP'), ('.', '.'), ('She', 'PRP'), ('says', 'VBZ'), (',', ','), ('``', '``'), ('I', 'PRP'), ('am', 'VBP'), ('Sue', 'NNP'), (',', ','), ('but', 'CC'), ('not', 'RB'), ('I', 'PRP'), ('am', 'VBP'), ('not', 'RB'), ('famous', 'JJ'), ('.', '.'), ("''", "''"), ('I', 'PRP'), ('get', 'VBP'), ('a', 'DT'), ('hot', 'JJ'), ('Italian', 

In [ ]:
# Apply the lemmatization function to the POS-tagged tokens
reviews_clean['Cleaned Review'] = reviews_clean['Cleaned Review'].apply(lemmatize)

# Display the DataFrame to check the lemmatized reviews
reviews_clean.head()

   Rating                 Date  \
0     5.0  2008-09-29 09:33:04   
1     3.0  2013-12-06 23:22:26   
2     5.0  2008-11-16 09:44:04   
3     5.0  2016-04-11 17:00:34   
4     4.0  2015-05-26 06:47:56   

                                              Review  \
0  its the best pizza hut i'v found in florida al...   
1  This place is an interesting combo. The chef, ...   
2  Pizza Hut is great! You get huge pizzas for yo...   
3  Always a great place to taste some tea with fr...   
4  Delish! Stopped in on a random Sunday and so g...   

                      Business          City State    Zip  \
0                    Pizza Hut       Orlando    FL  32819   
1  La Mongerie Bakery & Bistro       Atlanta    GA  30308   
2                    Pizza Hut       Orlando    FL  32819   
3                      Teavana       Atlanta    GA  30326   
4            Native Foods Cafe  Happy Valley    OR  97086   

   Business Rating Score                                     Cleaned Review  
0            

### Text normalization

I don't remove stopwords because we end up doing sentiment analysis

In [ ]:
def normalize_tokens(tokens):
  normalized_tokens = []

  for token in tokens:
      if isinstance(token, str):
        # Lowercasing
        token = token.lower()

        # Remove digits and punctuation
        token = re.sub(r'[\d\W]+', '', token)

        # If the token is not a stopword and its length is greater than 2, keep it
        if token and len(token) > 2:
            normalized_tokens.append(token)

  return normalized_tokens

In [ ]:
# Test the normalization
column = reviews_clean['Cleaned Review']
test = column[10]
print(test)
print("----------------------------------------------------------------------")
print(normalize_tokens(test))

['Shame', 'on', 'me', 'for', 'take', 'so', 'long', 'to', 'get', 'here', '!', 'I', 'walk', '&', 'notice', 'sign', 'say', ',', '``', 'Sue', 'say', '...', "''", 'I', 'be', 'already', 'like', 'this', 'place', '.', 'A', 'woman', 'take', 'my', 'order', '&', 'I', 'ask', 'if', 'she', 'be', 'the', 'famous', 'Sue', '.', 'She', 'say', ',', '``', 'I', 'be', 'Sue', ',', 'but', 'not', 'I', 'be', 'not', 'famous', '.', "''", 'I', 'get', 'a', 'hot', 'Italian', '.', 'It', 'be', 'perfect', '.', 'I', 'like', 'the', 'bun', 'a', 'little', 'toasted', 'so', 'it', 'do', 'not', 'get', 'soggy', 'with', 'the', 'oil', '.', 'I', 'will', 'be', 'back', '!']
----------------------------------------------------------------------
['shame', 'for', 'take', 'long', 'get', 'here', 'walk', 'notice', 'sign', 'say', 'sue', 'say', 'already', 'like', 'this', 'place', 'woman', 'take', 'order', 'ask', 'she', 'the', 'famous', 'sue', 'she', 'say', 'sue', 'but', 'not', 'not', 'famous', 'get', 'hot', 'italian', 'perfect', 'like', 'the

In [ ]:
# Apply normalization to the DataFrame
reviews_clean['Cleaned Review'] = reviews_clean['Cleaned Review'].apply(normalize_tokens)

In [ ]:
# Check normalization
reviews_clean.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[its, the, best, pizza, hut, find, florida, al..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[this, place, interesting, combo, the, chef, d..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[pizza, hut, great, you, get, huge, pizza, for..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[always, great, place, taste, some, tea, with,..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[delish, stopped, random, sunday, and, glad, n..."


### Remove rows if list of lemmas is empty

In [ ]:
# Function to remove rows where the array in 'Cleaned Review' is empty
def remove_empty_array_rows(df, column_name):
    filtered_df = df[df[column_name].apply(lambda x: len(x) > 0)]
    return filtered_df

In [ ]:
# Look at shape before filtering
reviews_clean.shape

(63230, 9)

In [ ]:
# Apply the function to the DataFrame
reviews_clean = remove_empty_array_rows(reviews_clean, 'Cleaned Review')

In [ ]:
# Look at shape after filtering
reviews_clean.shape

(63213, 9)

17 rows were filtered out

In [ ]:
# Check filtering
reviews_clean.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[its, the, best, pizza, hut, find, florida, al..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[this, place, interesting, combo, the, chef, d..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[pizza, hut, great, you, get, huge, pizza, for..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[always, great, place, taste, some, tea, with,..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[delish, stopped, random, sunday, and, glad, n..."


### Make cleaned string

In [ ]:
import ast  # Somewhere along my token list became a string (including apostrophes and all), so I first convert to tokens here

def tokens_to_string(tokens):
  tokens = ast.literal_eval(tokens)
  string = ' '.join(tokens)
  return string

In [ ]:
# Test the normalization
column = reviews_clean['Cleaned Review']
test = column[10]
print(type(test))
print(test)
print("----------------------------------------------------------------------")
print(tokens_to_string(test))

<class 'str'>
['shame', 'for', 'take', 'long', 'get', 'here', 'walk', 'notice', 'sign', 'say', 'sue', 'say', 'already', 'like', 'this', 'place', 'woman', 'take', 'order', 'ask', 'she', 'the', 'famous', 'sue', 'she', 'say', 'sue', 'but', 'not', 'not', 'famous', 'get', 'hot', 'italian', 'perfect', 'like', 'the', 'bun', 'little', 'toasted', 'not', 'get', 'soggy', 'with', 'the', 'oil', 'will', 'back']
----------------------------------------------------------------------
shame for take long get here walk notice sign say sue say already like this place woman take order ask she the famous sue she say sue but not not famous get hot italian perfect like the bun little toasted not get soggy with the oil will back


In [ ]:
# Apply normalization to the DataFrame
reviews_clean['Cleaned Review'] = reviews_clean['Cleaned Review'].apply(tokens_to_string)

In [ ]:
# Check filtering
reviews_clean.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...


### Section 1 Answer

Show the final number of rows in the cleaned dataframe

In [ ]:
# Look at shape of final dataframe
reviews_clean.shape

(63213, 9)

In [ ]:
%ls

'NLP Lab3.ipynb'   Restaurant_reviews_R.csv


In [ ]:
# Save dataframe
# Specify the filename
filename = 'cleaned_reviews.csv'

# Save the DataFrame as CSV
reviews_clean.to_csv(filename, index=False)

In [ ]:
%ls

 cleaned_reviews.csv   MyData/  'NLP Lab3.ipynb'   Restaurant_reviews_R.csv


## Section 2: Create Dataframe with FastText Embeddings as Features

### Load the Fasttext Pre-trained Model

The fasttext-wiki-news-subwords-300 is about 960MB and can be downloaded from: (https://drive.google.com/file/d/1E4Ag_QqobIAGZZaUeqAQxnw2CyCfNskK/view).

Download the file and save in under '/content/drive/MyDrive/Mydata/fasttext-wiki-news-subwords-300.gz'. You should save it under your own google drive folder, and use the left file navigation to copy the file path.

In [ ]:
%ls

drive/  sample_data/


In [ ]:
%cd drive/My Drive

/content/drive/My Drive


In [ ]:
%ls

'Academic Resources'/                               Official/
 Applications/                                     'ORNL 2024 Application Essay Questions (1).gdoc'
 Argonne/                                          'ORNL 2024 Application Essay Questions (2).gdoc'
'Bar Recommendation.gdoc'                          'Paper Ideas.gdoc'
 CGU/                                               response.tsv
'CGU Datathon.gdoc'                                 sample_info-2.csv
'Colab Notebooks'/                                  sample_info.csv
 ETC/                                              'Stat Learning'/
'HPO Workflow Image.gslides'                       'Swamp Date Night 3 9 (Responses) copy.gsheet'
'Machine Learning Club Statement of Purpose.gdoc'   table.txt
 Memories/                                          Talk.gdoc
 NLP/                                               UofR/
'Non-Parametric Stats'/                             Workouts/


In [ ]:
%cd NLP/Lab3

/content/drive/My Drive/NLP/Lab3


In [ ]:
%cd MyData

/content/drive/My Drive/NLP/Lab3/MyData


In [ ]:
%ls

 MyData/  'NLP Lab3.ipynb'   Restaurant_reviews_R.csv


In [ ]:
from gensim.models import KeyedVectors
# Load vectors directly from the .vec file
model = KeyedVectors.load_word2vec_format('fasttext-wiki-news-subwords-300.vec', binary=False)

### Read the Dataframe

In [ ]:
%cd ..

/content/drive/MyDrive/NLP/Lab3


In [ ]:
reviews_df = pd.read_csv('cleaned_reviews.csv')
reviews_df.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...


In [ ]:
reviews = reviews_df['Cleaned Review'].tolist()
print(reviews[0])

its the best pizza hut find florida always get great deal offer the staff very friendly make you feel welcome and you have child with you they make them feel extra special welcome them huge way too


### Create the 1% Token List

See Lab L3-TF-IDF Section 4.2

### Use gensim.corpora.Dictionary

The dfs object will count the number of times a term appears in a document. (The documentation: https://radimrehurek.com/gensim/corpora/dictionary.html)

In [ ]:
from gensim import corpora
from gensim.corpora import Dictionary

In [ ]:
corpus = [doc.split() for doc in reviews]
print(*corpus[0])

its the best pizza hut find florida always get great deal offer the staff very friendly make you feel welcome and you have child with you they make them feel extra special welcome them huge way too


In [ ]:
# Test making a dictionary
print(corpus[0])
dictionary_test = Dictionary([corpus[0]])
for id, word in dictionary_test.iteritems():
    print(f'ID: {id}, Word: {word}, Document Frequency: {dictionary_test.dfs[id]}')

['its', 'the', 'best', 'pizza', 'hut', 'find', 'florida', 'always', 'get', 'great', 'deal', 'offer', 'the', 'staff', 'very', 'friendly', 'make', 'you', 'feel', 'welcome', 'and', 'you', 'have', 'child', 'with', 'you', 'they', 'make', 'them', 'feel', 'extra', 'special', 'welcome', 'them', 'huge', 'way', 'too']
ID: 0, Word: always, Document Frequency: 1
ID: 1, Word: and, Document Frequency: 1
ID: 2, Word: best, Document Frequency: 1
ID: 3, Word: child, Document Frequency: 1
ID: 4, Word: deal, Document Frequency: 1
ID: 5, Word: extra, Document Frequency: 1
ID: 6, Word: feel, Document Frequency: 1
ID: 7, Word: find, Document Frequency: 1
ID: 8, Word: florida, Document Frequency: 1
ID: 9, Word: friendly, Document Frequency: 1
ID: 10, Word: get, Document Frequency: 1
ID: 11, Word: great, Document Frequency: 1
ID: 12, Word: have, Document Frequency: 1
ID: 13, Word: huge, Document Frequency: 1
ID: 14, Word: hut, Document Frequency: 1
ID: 15, Word: its, Document Frequency: 1
ID: 16, Word: make, 

In [ ]:
# Make actual dictionary
dictionary = Dictionary(corpus)

In [ ]:
# Sort the tokens by document frequency in descending order
sorted_tokens = sorted(dictionary.dfs.items(), key=lambda x: x[1], reverse=True)

# Print the top 5 most frequent tokens
print("Top 5 most frequent tokens:")
for id, freq in sorted_tokens[:5]:
    print(f'Word: {dictionary[id]}, Document Frequency: {freq}')

Top 5 most frequent tokens:
Word: the, Document Frequency: 57075
Word: and, Document Frequency: 55900
Word: have, Document Frequency: 40435
Word: not, Document Frequency: 38175
Word: for, Document Frequency: 36694


In [ ]:
print(len(corpus))

63213


In [ ]:
cutoff = len(corpus) * 0.01
print(cutoff)

632.13


In [ ]:
print(len(dictionary))

55552


In [ ]:
token_list = [dictionary.get(token_id) for token_id, count in dictionary.dfs.items() if count >= cutoff]
print(len(token_list))
print(token_list[:10])

842
['its', 'the', 'best', 'pizza', 'find', 'always', 'get', 'great', 'deal', 'offer']


### Section 2 Part C Answer

Output a dataframe snipet that shows a few of the top 1% words and their associated vectors. (see output in Lab L3-Word Embeddings Part II Section 4.2)

In [ ]:
# Filter tokens based on the FastText model's vocabulary (it's all of them)
filtered_tokens = [token for token in token_list if token in model.key_to_index]
print(len(filtered_tokens))
print(filtered_tokens[:10])

842
['its', 'the', 'best', 'pizza', 'find', 'always', 'get', 'great', 'deal', 'offer']


In [ ]:
data = {'Token': [], 'Vector_Snippet': []}
for token in filtered_tokens:
    if token in model:
        vector_snippet = model[token][:8]
        data['Token'].append(token)
        data['Vector_Snippet'].append(vector_snippet)

# Create a DataFrame
df = pd.DataFrame(data)
df.head()

,Token,Vector_Snippet
0,its,"[0.0228, -0.0294, 0.0318, 0.0323, 0.0068, -0.0..."
1,the,"[0.0129, 0.0026, 0.0098, 0.0063, 0.0102, -0.00..."
2,best,"[0.0009, 0.0115, -0.0071, 0.0139, 0.0053, -0.0..."
3,pizza,"[-0.0215, -0.0034, -0.0279, -0.0121, 0.0095, -..."
4,find,"[0.022, 0.0102, 0.0103, -0.0058, -0.0415, 0.00..."


### Create a Dataframe using word2vec vectors as features

See Lab L3-Word Embeddings Part II Section 4.3.

Note that, after you remove all non-vocabulary words from review_cleaned, you might end up with an empty review. If so you will need to address that because the Word2Vec model will not process an empty list.

In [ ]:
def average_word_vector(review, model, filtered_tokens):
    # Split the review string into tokens
    tokens = review.split()
    # Filter tokens to those in both the model's vocabulary and the list of filtered tokens (top 1%)
    vectors = [model[token] for token in tokens if token in model and token in filtered_tokens]
    # Compute average vector
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if the review contains no known/frequent words
        return np.zeros(model.vector_size)

In [ ]:
# Test embedding
column = reviews_df['Cleaned Review']
test = column[0]
print(type(test))
print(test)
print("----------------------------------------------------------------------")
print(average_word_vector(test, model, filtered_tokens)[:10])

<class 'str'>
its the best pizza hut find florida always get great deal offer the staff very friendly make you feel welcome and you have child with you they make them feel extra special welcome them huge way too
----------------------------------------------------------------------
[-0.00012353  0.00443529  0.01070882 -0.00187353 -0.00461765 -0.00279706
 -0.00316765 -0.06104706 -0.01742647 -0.00040294]


In [ ]:
# Compute the average word vectorization for each review and store it in a new column
reviews_df['FastText Embedding'] = reviews_df['Cleaned Review'].apply(lambda tokens: average_word_vector(tokens, model, filtered_tokens).tolist())

### Section 2 Answer

Output a snipet of the first 5 rows of the dataframe that includes the metadata and the word2vec feature vector for each review.

In [ ]:
reviews_df.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review,FastText Embedding
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...,"[-0.00012352918565738946, 0.004435293842107058..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...,"[-0.005872059613466263, -0.00390735175460577, ..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...,"[-0.006606060545891523, -0.0056939395144581795..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...,"[-0.001714285695925355, -0.006821428891271353,..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...,"[-0.0028229167219251394, 0.003914582543075085,..."


In [ ]:
# Save dataframe
# Specify the filename
filename = 'FastText_embedding.csv'

# Save the DataFrame as CSV
reviews_df.to_csv(filename, index=False)

## Section 3: Train Custom Word2Vec Embedding

See Lab L3-Word Embeddings Part II, Section "Part 3: Learn embeddings from scratch"

### Read the reviews

In [ ]:
%ls

 cleaned_reviews.csv      MyData/           Restaurant_reviews_R.csv
 FastText_embedding.csv  'NLP Lab3.ipynb'


In [ ]:
reviews_df = pd.read_csv('cleaned_reviews.csv')
reviews_df.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...


In [ ]:
reviews = reviews_df['Cleaned Review'].tolist()
print(type(reviews[0]))
print(reviews[0])

<class 'str'>
its the best pizza hut find florida always get great deal offer the staff very friendly make you feel welcome and you have child with you they make them feel extra special welcome them huge way too


### Create The Sentences

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
sentences = [txt.split() for txt in reviews]
print(sentences[0])

['its', 'the', 'best', 'pizza', 'hut', 'find', 'florida', 'always', 'get', 'great', 'deal', 'offer', 'the', 'staff', 'very', 'friendly', 'make', 'you', 'feel', 'welcome', 'and', 'you', 'have', 'child', 'with', 'you', 'they', 'make', 'them', 'feel', 'extra', 'special', 'welcome', 'them', 'huge', 'way', 'too']


### Train the Bigram Model

In [ ]:
# Train bigram
bigram = Phrases(sentences, min_count=30)

In [ ]:
bigram_phraser = Phraser(bigram)
sentences = bigram_phraser[sentences]

In [ ]:
bigrams = [word for sent in sentences for word in sent if '_' in word]
len(bigrams)

168794

In [ ]:
print(bigrams[:10])

['pizza_hut', 'feel_welcome', 'know_how', 'pizza_hut', 'your_money', 'customer_service', 'whole_foods', 'right_away', 'get_rid', 'man_who']


### Collect Frequent Words and Phrases

*Section 3 Part B Answer*

Output a list of the top twenty phrases and the top twenty words.

In [ ]:
from collections import Counter

# Count words and bigrams
word_counts = Counter()
bigram_counts = Counter()

In [ ]:
# Iterate with for loop catching both single and bigram frequencies
for sentence in sentences:
    for word in sentence:
        if '_' in word:  # bigrams can be identified by the underscore... this will need to be adjusted if other n-grams are present
            bigram_counts[word] += 1
        else:
            word_counts[word] += 1

In [ ]:
# Print top 20 words
print("Top 20 words:")
for word, count in word_counts.most_common(20):
    print(f"{word}: {count}")

# Print top 20 bigrams
print("\nTop 20 bigram phrases:")
for bigram, count in bigram_counts.most_common(20):
    print(f"{bigram}: {count}")

Top 20 words:
the: 332300
and: 225079
have: 102003
not: 89768
for: 75414
that: 56234
this: 55683
they: 53821
but: 53484
with: 50975
you: 48056
food: 44962
good: 40113
place: 37269
get: 35186
order: 33982
there: 27212
great: 24318
very: 23884
like: 22508

Top 20 bigram phrases:
come_back: 3896
first_time: 2691
customer_service: 2547
will_definitely: 2080
highly_recommend: 1780
more_than: 1734
next_time: 1667
ice_cream: 1503
our_server: 1362
even_though: 1308
every_time: 1285
make_sure: 1279
know_what: 1254
feel_like: 1237
check_out: 1173
happy_hour: 1152
too_much: 1151
thank_you: 1027
sit_down: 847
less_than: 839


### Learn the Custom Embedding

In [ ]:
from gensim.models import Word2Vec

# Train the model
custom_model = Word2Vec(sentences=sentences, vector_size=300, window=7, min_count=30, workers=2, epochs=20)

### Compare learned and pre-trained embedding

*Section 3 Part D Answer*

Print out the 10 most similar words for the 2 most frequent terms in the corpus and include the similarity scores. (See Lab L3-Word Embeddings Part II Section 3.4 for an example.)

*Section 3 Part E Answer*

Briefly discuss the differences you see between the output of the pre-trained model and the output of the custom embedding.

In [ ]:
top_words = word_counts.most_common(2)

for word, _ in top_words:
    print(f"Custom model most similar to {word}:")
    print(custom_model.wv.most_similar(word, topn=10))

    print(f"FastText model most similar to {word}:")
    if word in model:
        print(model.most_similar(word, topn=10))
    else:
        print("[Word not in FastText vocabulary]")


Custom model most similar to the:
[('and', 0.29696500301361084), ('nice', 0.2943045496940613), ('their', 0.2640668749809265), ('everything_else', 0.25482362508773804), ('inattentive', 0.25199541449546814), ('sparse', 0.24382518231868744), ('cons', 0.23486407101154327), ('disorganize', 0.23077861964702606), ('impressive', 0.22159942984580994), ('his', 0.22112740576267242)]
FastText model most similar to the:
[('its', 0.7545172572135925), ('first--the', 0.7535721659660339), ('them-the', 0.7452846765518188), ('their', 0.7397370934486389), ('you-the', 0.7390772104263306), ('his', 0.7357577681541443), ('this', 0.7341980338096619), ('this-the', 0.7223466038703918), ('all--the', 0.7222744822502136), ('in--the', 0.7206553816795349)]
Custom model most similar to and:
[('also', 0.3361510932445526), ('very', 0.3323914110660553), ('even_though', 0.331949383020401), ('with', 0.32299622893333435), ('the', 0.29696500301361084), ('which', 0.28433167934417725), ('when', 0.27260521054267883), ('super', 

In the FastText model, we only used the most common tokens while in the Word2Vec custom model we included bigrams. It makes sense, then, how the custom model has bigrams including the most common words as most similar while FastText does not.

Interestingly, the custom model seems to have lower similarity scores in general.

Something that surprised me is that FastText has embeddings for typos... this makes sense after some thought since there is still information even for typos, but was unexpected at first.

### Make Dataframe with Word2Vec embedding

In [ ]:
def average_word_vector(review, model, bigram_phraser):
    # Split the review string into tokens
    tokens = review.split()
    # Apply the bigram transformation to the tokens
    bigram_tokens = bigram_phraser[tokens]
    # Initialize an empty list to store vectors
    vectors = []
    for token in bigram_tokens:
        # Check if the token is in the model's vocabulary and append if it is
        if token in model.wv.key_to_index:
            vectors.append(model.wv[token])
    # Compute the average vector if there are any vectors in the list
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if the review contains no words in the review
        return np.zeros(model.vector_size)

In [ ]:
# Test embedding
column = reviews_df['Cleaned Review']
test = column[0]
print(type(test))
print(test)
print("----------------------------------------------------------------------")
print(average_word_vector(test, custom_model, bigram_phraser)[:10])

<class 'str'>
its the best pizza hut find florida always get great deal offer the staff very friendly make you feel welcome and you have child with you they make them feel extra special welcome them huge way too
----------------------------------------------------------------------
[ 0.4548119  -0.00923337 -0.12388884 -0.55302584 -0.07159236  0.29939815
  0.04257471 -0.2613649  -0.2668993  -0.15611586]


In [ ]:
reviews_df['Word2Vec Embedding'] = reviews_df['Cleaned Review'].apply(lambda review: average_word_vector(review, custom_model, bigram_phraser).tolist())

In [ ]:
reviews_df.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review,Word2Vec Embedding
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...,"[0.4548119008541107, -0.009233368560671806, -0..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...,"[0.4706699550151825, -0.11703094840049744, 0.0..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...,"[0.5399256348609924, -0.23302650451660156, -0...."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...,"[0.034546997398138046, 0.2712259888648987, -0...."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...,"[0.04914430156350136, 0.04304490238428116, 0.0..."


### Save File

In [ ]:
%ls

 cleaned_reviews.csv      MyData/           Restaurant_reviews_R.csv
 FastText_embedding.csv  'NLP Lab3.ipynb'


In [ ]:
# Save dataframe
# Specify the filename
filename = 'Word2Vec_embedding.csv'

# Save the DataFrame as CSV
reviews_df.to_csv(filename, index=False)

## Section 4 - Create document embedding features for aggregated reviews

After aggregation you should have 1075 rows. You can start with your answer to Section 2.

### Read the reviews with FastText embeddings

In [ ]:
%ls

 cleaned_reviews.csv      MyData/           Restaurant_reviews_R.csv
 FastText_embedding.csv  'NLP Lab3.ipynb'   Word2Vec_embedding.csv


In [ ]:
reviews_df = pd.read_csv('FastText_embedding.csv')
reviews_df.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review,FastText Embedding
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...,"[-0.00012352918565738946, 0.004435293842107058..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...,"[-0.005872059613466263, -0.00390735175460577, ..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...,"[-0.006606060545891523, -0.0056939395144581795..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...,"[-0.001714285695925355, -0.006821428891271353,..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...,"[-0.0028229167219251394, 0.003914582543075085,..."


### Convert string to list to be able to group

Not the most efficient that I've had multiple datatype conversions in this lab,
but with small(ish) datasets like we've had so far, we are fine.

In [ ]:
def string_to_list(string):
    try:
        # Use ast.literal_eval to convert string to list
        return ast.literal_eval(string)
    except ValueError:
        # Return an empty list or appropriate default if conversion fails
        return []

In [ ]:
# Convert 'FastText Embedding' column from string representation to lists
reviews_df['FastText Embedding'] = reviews_df['FastText Embedding'].apply(string_to_list)

### Aggregate by individual restaurant and calculate the mean document embedding.

In [ ]:
# Testing
column = reviews_df['FastText Embedding']
test = column[0]
print(test[:10])

[-0.00012352918565738946, 0.004435293842107058, 0.0107088228687644, -0.0018735297489911318, -0.004617647733539343, -0.002797059714794159, -0.003167646937072277, -0.061047062277793884, -0.017426470294594765, -0.00040294131031259894]


In [ ]:
# Custom aggregation function for FastText embeddings
def mean_embedding(embedding_lists):
    embeddings_array = np.array(embedding_lists)  # Convert the list of embeddings to a numpy array for easier calculation
    mean_embedding = np.mean(embeddings_array, axis=0)  # 0-axis for column-wise mean
    return mean_embedding.tolist()  # Convert back to list

In [ ]:
# Group reviews by business
business_df = reviews_df.groupby(['Business', 'City', 'State', 'Zip']).agg({
    'Business Rating Score': 'mean',
    'Review': lambda reviews: ' '.join(reviews),
    'FastText Embedding': mean_embedding
}).reset_index()

### Answer to Section 4

Snapshot of the first 5 rows of the aggregated restaurant dataframe with document vectors and original metadata.

In [ ]:
business_df.head()

,Business,City,State,Zip,Business Rating Score,Review,FastText Embedding
0,1/2 Roll,Vancouver,BC,V5V 3P6,3.0,The ambience is kind of funky & interesting. F...,"[5.077853497556027e-05, -0.0003010381067938664..."
1,21 Nickels,Watertown,MA,02472,4.0,I like 21 Nickels a ton and they'd attract mor...,"[3.8074714870288455e-05, -2.2126440006982663e-..."
2,24 Hour Pizza Delivery,Boston,MA,02124,1.0,This place needs to be shut down. I couldn't b...,"[-0.0001309090858381806, 1.8787880738576252e-0..."
3,3 Monkeys Pub & Grill,Vancouver,WA,98665,3.0,"Well, it's re-opened.\n\nStopped in today just...","[9.591835645996795e-05, -1.3990660331078938e-0..."
4,5th St Ribs-N-Blues,Atlanta,GA,30308,3.0,First visit here for my weekly southern BBQ fi...,"[2.6816639600677743e-06, -4.792387378128136e-0..."


In [ ]:
# Check shape
business_df.shape

(1075, 7)

### Save File

In [ ]:
%ls

 cleaned_reviews.csv      MyData/           Restaurant_reviews_R.csv
 FastText_embedding.csv  'NLP Lab3.ipynb'   Word2Vec_embedding.csv


In [ ]:
# Save dataframe
# Specify the filename
filename = 'business_FastText_embedding.csv'

# Save the DataFrame as CSV
business_df.to_csv(filename, index=False)

## Section 5: Sentiment Analysis

### Create Sentiment Analysis Scores for each Individual Review

TextBlob polarity and subjectivity, and Vader compound score.

### Read file

In [ ]:
%ls

 cleaned_reviews.csv      MyData/           Restaurant_reviews_R.csv
 FastText_embedding.csv  'NLP Lab3.ipynb'   Word2Vec_embedding.csv


In [ ]:
reviews_df = pd.read_csv('cleaned_reviews.csv')
reviews_df.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...


#### Create a new dataframe with just the columns we need

In [ ]:
reviews_df = reviews_df.drop(['Date', 'Rating', 'Review'], axis=1)

In [ ]:
reviews_df.head()

,Business,City,State,Zip,Business Rating Score,Cleaned Review
0,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...
1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...
2,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...
3,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...
4,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...


#### Add the polarity and subjectivity measures from TextBlob / compound score from NLTK

In [ ]:
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
# Instantiate the VADER sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

In [ ]:
# Function to apply sentiment analysis
def analyze_sentiment(review):
    # TextBlob scores
    blob = TextBlob(review)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity

    # NLTK VADER score
    nltk_score = sia.polarity_scores(review)['compound']

    return polarity, subjectivity, nltk_score

In [ ]:
# Apply function to each review and make columns for each metric
reviews_df['Sentiment Analysis'] = reviews_df['Cleaned Review'].apply(lambda x: analyze_sentiment(x))
reviews_df[['Polarity', 'Subjectivity', 'NLTK_Compound']] = pd.DataFrame(reviews_df['Sentiment Analysis'].tolist(), index=reviews_df.index)

In [ ]:
reviews_df.head()

,Business,City,State,Zip,Business Rating Score,Cleaned Review,Sentiment Analysis,Polarity,Subjectivity,NLTK_Compound
0,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut find florida always get...,"(0.5805803571428573, 0.6339285714285715, 0.9712)",0.580580,0.633929,0.9712
1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,this place interesting combo the chef david in...,"(0.2120748299319728, 0.47780612244897963, 0.9517)",0.212075,0.477806,0.9517
2,Pizza Hut,Orlando,FL,32819,3.5,pizza hut great you get huge pizza for your mo...,"(0.5802083333333333, 0.7625000000000001, 0.9636)",0.580208,0.762500,0.9636
3,Teavana,Atlanta,GA,30326,3.0,always great place taste some tea with friend ...,"(0.64375, 0.7, 0.9119)",0.643750,0.700000,0.9119
4,Native Foods Cafe,Happy Valley,OR,97086,4.5,delish stopped random sunday and glad normally...,"(0.2487179487179487, 0.6455128205128204, 0.9696)",0.248718,0.645513,0.9696


In [ ]:
reviews_df.shape

(63213, 10)

### Aggregate by Restaurant

And calculate the mean of the sentiment score.

In [ ]:
# Aggregate sentiment scores by business
business_sentiment_df = reviews_df.groupby(['Business', 'City', 'State', 'Zip', 'Business Rating Score']).agg({
    'Polarity': 'mean',
    'Subjectivity': 'mean',
    'NLTK_Compound': 'mean'
}).reset_index()

### Answer to Section 5 - The first five restaurants and their SA scores

In [ ]:
business_sentiment_df.head()

,Business,City,State,Zip,Business Rating Score,Polarity,Subjectivity,NLTK_Compound
0,1/2 Roll,Vancouver,BC,V5V 3P6,3.0,0.141160,0.486269,0.426003
1,21 Nickels,Watertown,MA,02472,4.0,0.243878,0.562022,0.797088
2,24 Hour Pizza Delivery,Boston,MA,02124,1.0,-0.118964,0.566542,-0.166997
3,3 Monkeys Pub & Grill,Vancouver,WA,98665,3.0,0.205975,0.566806,0.705704
4,5th St Ribs-N-Blues,Atlanta,GA,30308,3.0,0.212405,0.562440,0.652147


In [ ]:
business_sentiment_df.shape

(1075, 8)

### Save File

In [ ]:
%ls

 business_FastText.csv   FastText_embedding.csv  'NLP Lab3.ipynb'            Word2Vec_embedding.csv
 cleaned_reviews.csv     MyData/                  Restaurant_reviews_R.csv


In [ ]:
# Save dataframe
# Specify the filename
filename = 'business_FastText_sentiment.csv'

# Save the DataFrame as CSV
business_sentiment_df.to_csv(filename, index=False)

In [ ]:
%ls

 business_FastText.csv             FastText_embedding.csv   Restaurant_reviews_R.csv
 business_FastText_sentiment.csv   MyData/                  Word2Vec_embedding.csv
 cleaned_reviews.csv              'NLP Lab3.ipynb'
